# Insert triple for constituents related to objects

Run query:
```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?constituent ?object
WHERE{
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conXref .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
 	FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/1>}
      UNION {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/2>} 
      UNION {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/5>} .
  	?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

add triple

In [ ]:
import pandas

df = pandas.read_csv("conxref_object_link.csv").dropna(axis=1)
df.columns

In [ ]:
import rdflib, os
from rdflib import Graph, URIRef
from rdflib.namespace import SKOS

def insert_links(df):
    g = Graph()

    try:
        for _ , row in df.iterrows():
            conxrefs = URIRef(str(row[0]).strip().replace('"', ''))
            object = URIRef(str(row[1]).strip().replace('"', ''))
            g.add((conxrefs, SKOS.related, object))
        
    finally:
        print(len(g))
        return g

In [ ]:
insert_links(df).serialize(destination='conxreftoobject_graph.ttl')

# Insert triple for provenance activity (Acquisition)

run query:

```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT ?event ?constituent ?object
WHERE{
    ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/RoleTypeID> ?RoleTypeID .
    ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conxref .
  	?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
    FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/2> .
    ?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

add triples

In [3]:
import pandas

df = pandas.read_csv("provenance_activity.csv").dropna(axis=1)
df.columns

Index([' event', 'constituent', 'object'], dtype='object')

In [4]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g = Graph()
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):

            event = str(row[0]).strip().replace('"', '')
            constituent = URIRef(str(row[1]).strip().replace('"', ''))
            object = URIRef(str(row[2]).strip().replace('"', ''))

            # print(f"event:{event}\n constituent:{constituent}\n object: {object}\n")

            prov_activity = URIRef("www.example.com/Bronbeek/Provenance/"+str(i+1))
            acquisition = URIRef("www.example.com/Bronbeek/Acquisition/"+str(i+1))

            g.add((prov_activity, RDF.type, crm.E7_Activity))
            g.add((prov_activity, crm.P14_carried_out_by, constituent))
            g.add((prov_activity, crm.P2_has_type, URIRef(u"http://vocab.getty.edu/aat/300055863")))
            g.add((prov_activity, crm.P9_consists_of, acquisition))
            g.add((acquisition, RDF.type, crm.E8_Acquisition))
            g.add((acquisition, crm.P23_transferred_titled_from, constituent))
            g.add((acquisition, crm.P24_transferred_titled_of, object))
            g.add((acquisition, RDFS.label, Literal(event)))
            
        
    finally:
        print(len(g))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDF.type, crm.E55_Type))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDFS.label, Literal("Provenance Activity")))
        return g

In [5]:
insert_links(df).serialize(destination='provenance_activity.ttl')

0it [00:00, ?it/s]/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_50062/1378537723.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  event = str(row[0]).strip().replace('"', '')
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_50062/1378537723.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  constituent = URIRef(str(row[1]).strip().replace('"', ''))
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_50062/1378537723.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with D

1456336


<Graph identifier=N1cc78173eb8c454c9fcf88bf31e0796a (<class 'rdflib.graph.Graph'>)>

# Add triple for former owner

Run query:
```
SELECT ?event ?constituent ?object
WHERE{
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/RoleTypeID> ?RoleTypeID .
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConXrefID> ?conxrefID .
  	?conxrefID skos:related ?object .
    FILTER (?RoleTypeID = <https://example.com/RoleTypes/5>).
}
```

In [6]:
import pandas

df = pandas.read_csv("former_owner.csv").dropna(axis=1)
df.columns

Index([' event', 'constituent', 'object'], dtype='object')

In [7]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    g = Graph()
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):
            constituent = URIRef(str(row[1]).strip().replace('"', ''))
            object = URIRef(str(row[2]).strip().replace('"', ''))

            g.add((object, crm.P51_has_former_or_current_owner, constituent)) 
    finally:
        print(len(g))
        return g

In [8]:
insert_links(df).serialize(destination='former_owner.ttl')

0it [00:00, ?it/s]/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_50062/2434866988.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  constituent = URIRef(str(row[1]).strip().replace('"', ''))
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_50062/2434866988.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  object = URIRef(str(row[2]).strip().replace('"', ''))
69106it [00:06, 10373.93it/s]


34553


<Graph identifier=N7e573faebbfb4977b6e3848a9dbdde90 (<class 'rdflib.graph.Graph'>)>

# Add triple for person related to object (RoleTypeID=1)

Run query:
```
SELECT ?constituent ?object
WHERE{
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/RoleTypeID> ?RoleTypeID .
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConXrefID> ?conxrefID .
  	?conxrefID skos:related ?object .
    FILTER (?RoleTypeID = <https://example.com/RoleTypes/1>).
}
```

In [9]:
import pandas

df = pandas.read_csv("person_related.csv").dropna(axis=1)
df.columns

Index([' constituent', 'object'], dtype='object')

In [11]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    g = Graph()
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):
            constituent = URIRef(str(row[0]).strip().replace('"', ''))
            object = URIRef(str(row[1]).strip().replace('"', ''))

            g.add((object, SKOS.related, constituent)) 
    finally:
        print(len(g))
        return g

In [12]:
insert_links(df).serialize(destination='person_related.ttl')

0it [00:00, ?it/s]/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_2465/111733869.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  constituent = URIRef(str(row[0]).strip().replace('"', ''))
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_2465/111733869.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  object = URIRef(str(row[1]).strip().replace('"', ''))
26234it [00:02, 10046.15it/s]


13104


<Graph identifier=N4c321dd88ab7432e85b4956b81088bb8 (<class 'rdflib.graph.Graph'>)>